In [1]:
# 依存関係
'''
poetry add wheel

poetry add --group dev 'ipykernel' 'ipywidgets'

poetry add --group llm 'openvino'\
    "git+https://github.com/huggingface/optimum-intel.git"\
    "git+https://github.com/openvinotoolkit/nncf.git"
'''

'\npoetry add wheel\n\npoetry add --group dev \'ipykernel\' \'ipywidgets\'\n\npoetry add --group llm \'openvino\'    "git+https://github.com/huggingface/optimum-intel.git"    "git+https://github.com/openvinotoolkit/nncf.git"\n'

In [2]:
from pathlib import Path
import logging
import os
from datetime import datetime
from llm_config import LLM_MODELS_CONFIG

import openvino as ov
import nncf


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


In [3]:
## modelがgemmaの場合は、ログインが必要
def login_huggingface_hub():
    from huggingface_hub import notebook_login, whoami
    try:
        whoami()
        print('Authorization token already provided')
    except OSError:
        notebook_login()
        print('Authorization token has been provided')

In [4]:
nncf.set_log_level(logging.ERROR)

core = ov.Core()

# デバイスの特定
support_devices = core.available_devices
device = 'NPU' if 'NPU' in support_devices else 'CPU'


# ダウンロードするモデル
# NPUならPhi-3-mini-4k-instruct, CPUならgemma-2b-it
model_id = "microsoft/Phi-3-mini-4k-instruct" if device == 'NPU' else "google/gemma-2b-it"

## modelがgemmaの場合は、HuggingFaceのログインが必要
if model_id == "google/gemma-2b-it":
    login_huggingface_hub()

model_name = model_id.split('/')[-1]

remote_code = LLM_MODELS_CONFIG[model_name]['remote_code']  # Phi-3はTrue

# モデルを保存するディレクトリ
model_dir = Path(f'../../model/{model_name}')
fp16_model_dir = model_dir / "FP16"  # float 16bitモデルの保存先
int8_model_dir = model_dir / "INT8"  # 量子化モデルの保存先(8bit)
int4_model_dir = model_dir / "INT4"  # 量子化モデルの保存先(4bit)

In [6]:
# NPUの要件がINT8以上なので、コメントアウト

# INT4の設定
compression_configs = {
    # ここのパラメータは要調整
    # "sym":          対称量子化の利用
    # 'group_size':  グループサイズ  (64, 128が無難？)
    # 'ratio':       量子化後のパラメータの割合  (0.5~0.8で試す)
    "gemma-2b-it": {
        "sym": True,
        "group_size": 64,
        "ratio": 0.6,
    },
    "default": {
        "sym": False,
        "group_size": 128,
        "ratio": 0.8,
    },
}

In [7]:
model_id

'google/gemma-2b-it'

In [9]:
# optimum-cliでモデルをopenvino形式でダウンロード
export_command_base = "optimum-cli export openvino --model {} --task text-generation-with-past".format(model_id)

def convert_to_fp16():
    global export_command_base
    export_command = ''
    # すでに存在する場合はスキップ
    if (fp16_model_dir / "openvino_model.xml" ).exists():
        return
    if remote_code:
        # Phi-3のみ
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " --weight-format fp16"
    export_command += " " + str(fp16_model_dir)
    # モデルのダウンロード開始時間
    start_model_download = datetime.now()
    print('export_command:', export_command)
    os.system(export_command)  # モデルのダウンロード
    # モデルのダウンロード終了時間
    end_model_download = datetime.now() - start_model_download
    print('export done', end_model_download.total_seconds())


def convert_to_int8():
    global export_command_base
    if (int8_model_dir / "openvino_model.xml").exists():
        return
    if remote_code:
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " --weight-format int8"
    export_command += " " + str(int8_model_dir)
    # モデルのダウンロード開始時間
    start_model_download = datetime.now()
    print('export_command:', export_command)
    os.system(export_command)  # モデルのダウンロード
    # モデルのダウンロード終了時間
    end_model_download = datetime.now() - start_model_download
    print('export done', end_model_download.total_seconds())


# NPUの要件がINT8以上なので、コメントアウト
def convert_to_int4():
    global export_command_base
    if (int4_model_dir / "openvino_model.xml").exists():
        return
    if remote_code:
        export_command_base += " --trust-remote-code"
    # 量子化の設定
    model_compression_params  = compression_configs.get(model_name, compression_configs["default"])
    export_command = export_command_base + " --weight-format int4"
    int4_compression_args = " --group-size {} --ratio {}".format(model_compression_params["group_size"], model_compression_params["ratio"])
    if model_compression_params["sym"]:
        int4_compression_args += " --sym"
    export_command += int4_compression_args + " " + str(int4_model_dir)
    # モデルのダウンロード開始時間
    start_model_download = datetime.now()
    print('export_command:', export_command)
    os.system(export_command)  # モデルのダウンロード
    # モデルのダウンロード終了時間
    end_model_download = datetime.now() - start_model_download
    print('export done', end_model_download.total_seconds())


In [10]:
# NPUの要件がINT8以上なので、コメントアウト
convert_to_int4()

In [11]:
convert_to_int8()

export_command: optimum-cli export openvino --model google/gemma-2b-it --task text-generation-with-past --weight-format int8 ../../model/gemma-2b-it/INT8
INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


/Users/yamashitaryuunosuke/TECH.C./コンテスト/OpenVINO2024/Taltner/server/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Framework not specified. Using pt to export the model.
Trying to resume download...
Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/07/bd/07bd0ee7b469afc0b9115a5c9fb06966403948914ff05454f8f9695190fa5cbc/561656f892a2a1ca0837ca529c5ce820a72b40f4f563b1cd0a1acc0b3899c30c?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model-00001-of-00002.safetensors%3B+filename%3D%22model-00001-of-00002.safetensors%22%3B&Expires=1716704836&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjcwNDgzNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzA3L2JkLzA3

In [15]:
convert_to_fp16()

In [12]:
# モデルが保存されているディレクトリのサイズを確認

fp16_weights = fp16_model_dir / "openvino_model.bin"
int8_weights = int8_model_dir / "openvino_model.bin"
int4_weights = int4_model_dir / "openvino_model.bin"

if fp16_weights.exists():
    print(f"Size of FP16 model is {fp16_weights.stat().st_size / 1024 / 1024:.2f} MB")

for precision, compressed_weights in zip([8, 4], [int8_weights, int4_weights]):
    if compressed_weights.exists():
        print(f"Size of model with INT{precision} compressed weights is {compressed_weights.stat().st_size / 1024 / 1024:.2f} MB")
    if compressed_weights.exists() and fp16_weights.exists():
        print(f"Compression rate for INT{precision} model: {fp16_weights.stat().st_size / compressed_weights.stat().st_size:.3f}")

Size of FP16 model is 7288.13 MB
Size of model with INT4 compressed weights is 2336.74 MB
Compression rate for INT4 model: 3.119


In [13]:
# デバイスの選択
support_devices = core.available_devices
device = 'NPU' if 'NPU' in support_devices else 'CPU'  # 実機のNPUが使えればいいのだけれど。。。

In [14]:
device

'CPU'

In [15]:
from ipywidgets import widgets

available_models = []
if int4_model_dir.exists():
    available_models.append("INT4")
if int8_model_dir.exists():
    available_models.append("INT8")
if fp16_model_dir.exists():
    available_models.append("FP16")

model_to_run = widgets.Dropdown(
    options=available_models,
    value=available_models[0],
    description="Model to run:",
    disabled=False,
)

model_to_run

Dropdown(description='Model to run:', options=('INT4', 'FP16'), value='INT4')

In [16]:
model_to_run.value

'INT4'

In [17]:
from transformers import AutoConfig, AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM

if model_to_run.value == "INT4":  # 4bitモデルを使う場合
    model_dir = int4_model_dir
elif model_to_run.value == "INT8":  # 8bitモデルを使う場合
    model_dir = int8_model_dir
else:
    model_dir = fp16_model_dir  # 16bitモデルを使う場合
print(f"Loading model from {model_dir}")

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

tok = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

ov_model = OVModelForCausalLM.from_pretrained(
    model_dir,
    device=device,
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading model from ../../model/Phi-3-mini-4k-instruct/INT4


/home/korryu/Documents/TECH.C/Taltner/server/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The argument `trust_remote_code` is to be used along with export=True. It will be ignored.
Compiling the model to CPU ...


In [24]:
# モデルの動作確認
tokenizer_kwargs = LLM_MODELS_CONFIG[model_name].get("tokenizer_kwargs", {})

SYSTEM_PROMPT = '''\
# 指示
あなたは、ユーザーが初対面の相手との会話を補助するためのアシスタントです。
ユーザーと相手の会話を円滑に進めるために、ユーザーの補佐をします。
以下の要素を考慮して、4つの話題をJSONフォーマットで提示しなさい。

# 条件
1. 初対面の相手との会話を始めるのに適した話題の提供
2. 会話の流れを考慮して、次の話題を提示
3. 単語くらいの短い文章で話題を提示
4. 出力フォーマットは{{JSON}}形式

以下のフォーマットで出力してください：

```json
{
  "topics": [
    "話題1",
    "話題2",
    "話題3",
    "話題4"
  ]
}
```

# 例1
```json
{
  "topics": [
    "好きな食べ物",
    "どこ出身",
    "好きなゲーム",
    "最近見た映画"
  ]
}
```

# 例2
```json
{
  "topics": [
    "趣味",
    "休日の過ごし方",
    "好きな音楽",
    "行ってみたい場所"
  ]
}
```

# 例3
```json
{
  "topics": [
    "好きなスポーツ",
    "好きなアニメ",
    "好きな漫画",
    "好きな映画"
  ]
}
```\
'''

conversation = '''\
自分: 「初めまして〜。俺AI専攻の人と話すの初めてなんだけど”AI専攻ってどんなことしてるの”？」

相手: 「AI専攻は機械学習とか画像や音声認識とかLLMについて勉強してるよ」
'''

start_message: str = LLM_MODELS_CONFIG[model_name]['start_message']
prompt_template: str = LLM_MODELS_CONFIG[model_name]['prompt_template']

sys_prompt = start_message.format(
    SYSTEM_PROMPT=SYSTEM_PROMPT
)

prompt = sys_prompt + prompt_template.format(
    user=conversation
)

input_tokens = tok(prompt, return_tensors="pt", **tokenizer_kwargs)

answer = ov_model.generate(
    **input_tokens,
    max_new_tokens=100,
    temperature=0.7,
    do_sample=True
)

ans = tok.batch_decode(answer, skip_special_tokens=True)[0]
print(ans)

# 指示
あなたは、ユーザーが初対面の相手との会話を補助するためのアシスタントです。
ユーザーと相手の会話を円滑に進めるために、ユーザーの補佐をします。
以下の要素を考慮して、4つの話題をJSONフォーマットで提示しなさい。

# 条件
1. 初対面の相手との会話を始めるのに適した話題の提供
2. 会話の流れを考慮して、次の話題を提示
3. 単語くらいの短い文章で話題を提示
4. 出力フォーマットは{{JSON}}形式

以下のフォーマットで出力してください：

```json
{
  "topics": [
    "話題1",
    "話題2",
    "話題3",
    "話題4"
  ]
}
```

# 例1
```json
{
  "topics": [
    "好きな食べ物",
    "どこ出身",
    "好きなゲーム",
    "最近見た映画"
  ]
}
```

# 例2
```json
{
  "topics": [
    "趣味",
    "休日の過ごし方",
    "好きな音楽",
    "行ってみたい場所"
  ]
}
```

# 例3
```json
{
  "topics": [
    "好きなスポーツ",
    "好きなアニメ",
    "好きな漫画",
    "好きな映画"
  ]
}
``` 自分: 「初めまして〜。俺AI専攻の人と話すの初めてなんだけど”AI専攻ってどんなことしてるの”？」

相手: 「AI専攻は機械学習とか画像や音声認識とかLLMについて勉強してるよ」
 ```json
{
  "topics": [
    "AIにおける最近の進展",
    "機械学習の基本的な概念",

    "画像認識の応用例",

    "音声認識の革新的な進展"
  ]
}
```
